# Programmieraufgabe IV     ( 6 + 10 + 4 = 20 Punkte) 
## Abgabe in der Woche 17. bis 21. Dezember

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import numpy.linalg as la

## Teilaufgabe a) 

Schreiben Sie eine Funktion, die eine gegebene Matrix $A$ (numpy-array) mittels Householder-Transformationen in eine obere Hessenbergmatrix überführt. 


In [2]:
# Das ist die Kopie von altem Code

def householder(arg, w): 
    
    ## Diese Funktion soll eine Householder-Transformation $I - 2ww^T$ auf das Argument arg (Matrix oder Vektor) anwenden.
    
    l = len(w)
    
    H_ = np.identity(l)
    for i in range(l):
        for j in range(l):
            H_[i][j] -= 2*w[i]*w[j]
    
    (num_rows, num_cols) = np.shape(arg)
    
    H = np.identity(num_rows)
    H[num_rows-l:, num_rows-l:] = H_
    
    result = H@arg
    
    return (result, H)

In [3]:
# Diese Funktion funktioniert noch nicht.
# Die Dimensionen stimmen nicht ueberein.

def intohessenberg(A):
    
    m = np.shape(A)[0] - 2
    
    for k in range(m):
        
        v = np.copy(A[...,k][k+1:]) # A[k+1:m][k]
        v[0] += np.sign(v[0]) * np.linalg.norm(v)
        v = v / np.linalg.norm(v)
        
        ## Benutzte entweder die alte Funktion von oben:
        A,_ = householder(A,v)
        
        ## Debug output
        # print("k =", k)
        # print("v =", v)
        # print("np.transpose(v) =", np.transpose(v))
        # print("A[...,k:][k+1:] =", A[...,k:][k+1:])
        # print("np.transpose(v) @ A[...,k:][k+1:] =", np.transpose(v) @ A[...,k:][k+1:])
        # print("v @ (np.transpose(v) @ A[...,k:][k+1:]) =", v @ (np.transpose(v) @ A[...,k:][k+1:]))
        
        ## Oder schreibe die Funktion neu:
        # A[...,k:][k+1:] -= 2 * (v @ (np.transpose(v) @ A[...,k:][k+1:])) # A[k+1:m][k:m]
        # A[...,k+1:][k:] -= 2 * ((A[...,k+1:][k:] @ v) @ np.transpose(v)) # A[1:m][k+1:m]
        
        # print("")
        
    return A

In [4]:
A = [[1,2,3],[4,5,6],[7,8,9],[10,11,12],[13,14,15],[16,17,18],[19,20,21]]
A = np.array(A).astype(float)

print(intohessenberg(A))

IndexError: index 3 is out of bounds for axis 1 with size 3

## Teilaufgabe b)

Implementieren Sie das in der Vorlesung behandelte QR-Verfahren zur Eigenwertbestimmung einer oberen Hessenbergmatrix $A$ (numpy-array). Input der Funktion soll die Matrix $A$, eine maximale Anzahl von Iterationen $\text{itmax}$ und eine Toleranz $\text{tol}$ sein, die in ein (sinnvolles) Abbruchkriterium eingeht.

Ausgegeben werden soll ein array mit den Näherungen für die Eigenwerte, geordnet nach Größe. Außerdem soll eine Liste der Eigenwertnäherungen aller Iterationen ausgegeben werden. 

Beachten Sie, dass hier die __in der Vorlesung besprochene praktische Umsetzung des QR-Verfahrens zur Eigenwertbestimmung__ implementiert werden soll, d.h. ein bloßes Ausrechnen der QR-Zerlegung $A=QR$ und anschließende Zuweisung $A \leftarrow RQ$, explizites Aufstellen und Ausmultiplizieren von Givens-Rotations-Matrizen etc. soll hier _nicht_ verwendet werden!

In [5]:
def qr_eigenvalues(A, itmax, tol):
    
    ## to be completed ## 
    
    return eigvals, eigvalslist

Testen Sie Ihre Implementierung, indem Sie Ihre Ergebnisse für eine zufällige symmetrische $10 \times 10$-Matrix und die übliche Matrix $A = \text{tridiag}(1,4,1) \in \mathbb{R}^{5 \times 5}$ mit der in numpy vorimplementierten Eigenwertberechnung (wählen Sie eine geeignete Routine) vergleichen. 

In [6]:
scal = 10 # Skaliere die Zufallszahlen -> [-scal,scal)

# np.random.random_sample((m,n)) gives B \in [0,1)^(m x n)
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.random.html
B = 2*scal*(np.random.random_sample((10,10)) - .5)
B = (B + B.T)/2

A = [[4,1,0,0,0],
     [1,4,1,0,0],
     [0,1,4,1,0],
     [0,0,1,4,1],
     [0,0,0,1,4]]
A = np.array(A).astype(float)

print(B)
print(A)

[[ 0.93344224  5.70611355  0.26229371 -2.49815945 -7.00440232 -3.07894994
   7.91266884  3.76627965  8.18782569  0.57987203]
 [ 5.70611355  4.15214067  2.45190665 -8.15350065 -1.16229169 -3.88880774
  -3.16917207 -0.11802294 -5.14140572  0.61040095]
 [ 0.26229371  2.45190665  9.33502082  4.58803392  1.39384624  3.57260246
   2.43151323 -6.63885055  2.44482567 -3.23732847]
 [-2.49815945 -8.15350065  4.58803392 -8.7868998  -1.03644557  0.65108626
  -2.79151023  7.94958499  3.61211208 -0.55072857]
 [-7.00440232 -1.16229169  1.39384624 -1.03644557  4.87868941 -5.59200435
   2.68760723  2.97224254 -1.77452649  3.46647758]
 [-3.07894994 -3.88880774  3.57260246  0.65108626 -5.59200435  8.42113935
  -1.5824998   0.40243775 -0.41162907  6.43339099]
 [ 7.91266884 -3.16917207  2.43151323 -2.79151023  2.68760723 -1.5824998
  -6.46961839 -0.10162352  1.66486983  0.27063798]
 [ 3.76627965 -0.11802294 -6.63885055  7.94958499  2.97224254  0.40243775
  -0.10162352  1.68183395  0.7195351  -7.76489705]
 

In [7]:
itmax = 100
tol = .001

myeigvalsA, _ = qr_eigenvalues(A, itmax, tol)
myeigvalsB, _ = qr_eigenvalues(B, itmax, tol)

NameError: name 'eigvals' is not defined

In [8]:
# The column v[:,i] is the eigenvector corresponding to the eigenvalue w[i]
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.eig.html

w1, v1 = la.eig(A)
w2, v2 = la.eig(B)
eigvalsA = w1
eigvalsB = w2

print("A:")
for i in range(w1.shape[0]):
    print("w_" + str(i), "=", w1[i])
    print("v_" + str(i), "=", v1[:,i])
print("B:")
for i in range(w2.shape[0]):
    print("w_" + str(i), "=", w2[i])
    print("v_" + str(i), "=", v2[:,i])

A:
w_0 = 2.26794919243
v_0 = [-0.28867513  0.5        -0.57735027  0.5        -0.28867513]
w_1 = 3.0
v_1 = [ -5.00000000e-01   5.00000000e-01  -3.42143760e-16  -5.00000000e-01
   5.00000000e-01]
w_2 = 4.0
v_2 = [  5.77350269e-01  -4.90435668e-17  -5.77350269e-01  -4.10676918e-16
   5.77350269e-01]
w_3 = 5.73205080757
v_3 = [ 0.28867513  0.5         0.57735027  0.5         0.28867513]
w_4 = 5.0
v_4 = [ -5.00000000e-01  -5.00000000e-01   5.56426531e-16   5.00000000e-01
   5.00000000e-01]
B:
w_0 = -20.9382107977
v_0 = [-0.11170093 -0.28508645  0.30047551 -0.64996095 -0.20261353 -0.18716728
 -0.15827023  0.46058596 -0.03341947  0.28129037]
w_1 = -16.8338069607
v_1 = [ 0.60359855 -0.29849763  0.02681262 -0.07922697  0.29544804  0.07804989
 -0.61723348 -0.16903431 -0.16339002 -0.09972323]
w_2 = -9.61123881034
v_2 = [-0.19768124  0.07452563 -0.13267378 -0.5380617   0.21265283  0.29877478
 -0.02953136 -0.1035554   0.45183382 -0.54591035]
w_3 = -1.76427453727
v_3 = [-0.28059351  0.4026825   0.0

## Teilaufgabe c) 

Erstellen Sie einen Plot, aus dem für jeden Eigenwert der Matrix die Konvergenzgeschwindigkeit der jeweiligen Eigenwertnäherungen des QR-Verfahrens hervorgeht. Als die "richtigen" Eigenwerte können Sie beispielsweise die mit einer numpy-Routine berechneten Eigenwerte oder die letzte Iterierte Ihres Verfahrens verwenden. 

Test-Matrix soll $B = B(a,b,c,d)$ wie unten angegeben sein. 

Variieren Sie die Parameter $a,b,c,d$ und interpretieren Sie Ihre Beobachtungen.

In [9]:
a = 1
b = 2 
c = 3
d = 4

T = 0.5*np.array([[1,1,1,1],[1,1,-1,-1],[-1,1,-1,1],[-1,1,1,-1]])
B = np.diag([a, b, c, d])
B = T.T @ (B @ T) # np.matmul(T.T, np.matmul(B, T))